In [13]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))
sys.path.append(str(project_root/'src'))

In [14]:
project_root

PosixPath('/cluster/tufts/liulab/jboerm01/CS150-FinalProject')

In [15]:
from pathlib import Path

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from torch.utils.data import Dataset
import glob

import os, torch

In [17]:
DATASET_PATH = Path("/cluster/tufts/liulab/data")

SAMPLE_DIR = Path("/cluster/tufts/liulab/jboerm01/CS150-FinalProject/samples")
SAMPLE_DIR.mkdir(exist_ok=True)

In [34]:
HARVEY_MODEL_DIR = Path("/cluster/tufts/hugheslab/eharve06/CS150-FinalProject/experiments/")

BOERMA_MODEL_DIR = Path("/cluster/tufts/liulab/jboerm01/CS150-FinalProject/models")
BOERMA_MODEL_DIR.mkdir(exist_ok=True)

MNIST_HARVEY_BEST = 'MNIST_HARVEY_BEST'
MODELS = {
    MNIST_HARVEY_BEST: HARVEY_MODEL_DIR / "MNIST_UpscalingDDPM2_noise=low_T=5/batch_size=32_epochs=1000_loss=simple_lr=0.01_random_state=42.pth"
}

## Generate Samples

In [46]:
## Keep all in memory for now
class ChunkedTensorDataset(Dataset):
    def __init__(self, directory: Path):
        self.files = sorted(directory.glob("*.pt"))
        assert len(self.files) > 0, "No .pt files found"

        self.chunks = []
        self.cum_sizes = []

        total = 0
        for f in self.files:
            chunk = torch.load(f, map_location="cpu")
            self.chunks.append(chunk)
            total += chunk.size(0)
            self.cum_sizes.append(total)

    def __len__(self):
        return self.cum_sizes[-1]

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            # Handle slice
            start, stop, step = idx.indices(len(self))
            result = []
            for i in range(start, stop, step):
                result.append(self._get_single(i)[0])
            return torch.stack(result), 0  # dummy label
        else:
            return self._get_single(idx)

    def _get_single(self, idx):
        # Return a single item
        for i, cs in enumerate(self.cum_sizes):
            if idx < cs:
                prev = 0 if i == 0 else self.cum_sizes[i - 1]
                return self.chunks[i][idx - prev], 0  # dummy label
        raise IndexError

In [20]:
class ChunkedSampleWriter:
    def __init__(self, out_dir, chunk_size=5000):
        self.out_dir = out_dir
        self.chunk_size = chunk_size
        os.makedirs(out_dir, exist_ok=True)

        self.buffer = []
        self.buffer_count = 0
        self.chunk_idx = 0
        self.total_saved = 0

    def add(self, samples: torch.Tensor):
        """
        samples: Tensor [B, C, H, W] on CPU
        """
        assert samples.device.type == "cpu"
        assert samples.dim() == 4

        self.buffer.append(samples)
        self.buffer_count += samples.size(0)

        if self.buffer_count >= self.chunk_size:
            self._flush()

    def _flush(self):
        chunk = torch.cat(self.buffer, dim=0)

        to_save = chunk[: self.chunk_size]
        remainder = chunk[self.chunk_size :]

        path = os.path.join(
            self.out_dir, f"samples_{self.chunk_idx:04d}.pt"
        )
        torch.save(to_save, path)

        self.total_saved += to_save.size(0)
        self.chunk_idx += 1

        # reset buffer with remainder
        self.buffer = [remainder] if remainder.numel() > 0 else []
        self.buffer_count = remainder.size(0) if remainder.numel() > 0 else 0

    def finalize(self):
        """
        Save remaining samples (< chunk_size)
        """
        if self.buffer_count > 0:
            chunk = torch.cat(self.buffer, dim=0)
            path = os.path.join(
                self.out_dir, f"samples_{self.chunk_idx:04d}.pt"
            )
            torch.save(chunk, path)

            self.total_saved += chunk.size(0)
            self.chunk_idx += 1

            self.buffer = []
            self.buffer_count = 0

In [21]:
writer = ChunkedSampleWriter(
    out_dir="generated/model_A",
    chunk_size=5000,
)

In [22]:
from src.models import UpscalingDDPM
from src.unet import UNet, UpscaleUNet

### Sanity Check

In [ ]:
eps_model = UNet(image_channels=1)
gen_model = UpscalingDDPM(eps_model=eps_model, T=5)
best_weights = torch.load(MODELS[MNIST_HARVEY_BEST], map_location="cpu", weights_only=True)
gen_model.load_state_dict(best_weights)

<All keys matched successfully>

In [24]:
def sample_upscalingddpm(model: UpscalingDDPM, batch_size=36):
    x_T = 0.1309 + 0.2892 * torch.randn(size=(batch_size, 1, 1, 1))
    #x_T = torch.randn(size=(batch_size, 1, 1, 1))
    x_ts = [x_T]

    ts = torch.arange(start=0, end=model.T)
    alphas = 1.0 - model.betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

    model.eval()

    with torch.no_grad():
        
        for t in reversed(ts):

            x_t = torch.nn.functional.interpolate(x_ts[-1], scale_factor=2, mode="nearest")
            
            logits = model.eps_model(x_t, torch.full((batch_size,), t))
            mean = sqrt_recip_alphas[t] * (x_t - model.betas[t] / sqrt_one_minus_alphas_cumprod[t] * logits)
            
            if t > 0:
                noise = torch.randn_like(mean)   
                sigma = torch.sqrt(model.betas[t] * (1.0 - alphas_cumprod[t - 1]) / (1.0 - alphas_cumprod[t]))
                x_ts.append(mean + sigma * noise)
                #x_ts.append(mean)
            else:
                x_ts.append(mean)
    
    return list(reversed(x_ts))

In [25]:
x_ts = sample_upscalingddpm(gen_model)

In [26]:
len(x_ts)

6

In [27]:
from matplotlib import pyplot as plt

def display_samples(x_t):
    ncols, nrows = 6, 6
    fig, axs = plt.subplots(dpi=300, figsize=(4*ncols, 4*nrows), ncols=ncols, nrows=nrows)

    for i in range(6):
        for j in range(6):
        
            axs[i,j].imshow(x_t[(i*6)+j].permute(1, 2, 0), cmap="gray", vmin=x_t[(i*6)+j].min(), vmax=x_t[(i*6)+j].max())
            axs[i,j].axis("off")

    fig.tight_layout()

    plt.show()

In [28]:
display_samples(x_ts[0])

### Generate For Metrics

First, dump samples for each time for the model

In [29]:
from tqdm.auto import tqdm

In [37]:
def get_sample_dir(model_name: str): 
    return SAMPLE_DIR / model_name

In [60]:
def dump_gen_samples(model_name: str, image_channels: int, n_samples = 60_000, batch_size=50):
    eps_model = UNet(image_channels=image_channels)
    gen_model = UpscalingDDPM(eps_model=eps_model, T=5)

    model_path = MODELS[model_name]

    best_weights = torch.load(model_path, map_location="cpu", weights_only=True)
    gen_model.load_state_dict(best_weights)

    gen_model.eval()

    # test batch - list of xt_BCHW by time
    with torch.no_grad():
        samples_ls = sample_upscalingddpm(gen_model, batch_size=1)
    n_timesteps = len(samples_ls)
    
    # each timestep gets its own folder. samples aligned by timestep
    sample_dir = get_sample_dir(model_name)
    sample_dir.mkdir(exist_ok=True)
    writers = [
        ChunkedSampleWriter(sample_dir / f"{t}", 3000) 
        for t in range(n_timesteps)
    ]

    # now save samples
    n_batches_total = n_samples // batch_size + 1
    for _ in tqdm(
        range(n_batches_total),
        desc="Generating samples",
        leave=True,
    ):
        with torch.no_grad():
            samples_ls = sample_upscalingddpm(gen_model, batch_size=batch_size)
        for sample_t, writer_t in zip(samples_ls, writers): 
            # quantize for data storage
            samples_uint8 = torch.round(torch.clamp(sample_t, 0, 1) * 255).to(torch.uint8)
            writer_t.add(samples_uint8)

    for writer in writers: 
        writer.finalize()


### BIG COMPUTE - Can put to a job

In [ ]:
dump_gen_samples(MNIST_HARVEY_BEST, 1, 60_000, 100)

Generating samples:   0%|          | 0/601 [00:00<?, ?it/s]

Check!

In [47]:
sample_dir_0 = get_sample_dir(MNIST_HARVEY_BEST) / "0"
dataset = ChunkedTensorDataset(sample_dir_0)

/tmp/ipykernel_2287162/17780822.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chunk = torch.load(f, map_location="cpu")


In [ ]:
dataset[:50][0]

torch.Size([50, 1, 32, 32])

In [58]:
display_samples(dataset[:50][0])

In [59]:
display_samples(torch.round(torch.clamp(dataset[:50][0], 0, 1) * 255) / 255)

### Check Dataset

## Compute Metrics for Models

### MNIST-FID
The "FID-Statistic" is computed by the deepest layer in the Inception v3
classifier. 

To compute FID between datasets, start by computing all FID-Statistics. We then 
fit a Gaussian to the FID-Statistic vectors for dataset A, and do the same for 
dataset B. The FID is the wasserstein distance between the gaussians, 
which can be computed in closed-form.

#### Training MNIST Classifier

In [ ]:
### Cell generated by AI
class MNISTClassifier(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()

        # Feature extractor
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),  # 28x28 → 28x28
            nn.ReLU(),
            nn.MaxPool2d(2),                          # 14x14
            nn.Conv2d(32, 64, 3, stride=1, padding=1),# 14x14
            nn.ReLU(),
            nn.MaxPool2d(2),                          # 7x7
        )

        self.fc_embed = nn.Linear(64 * 7 * 7, embedding_dim)
        self.fc_out   = nn.Linear(embedding_dim, 10)

    def forward_features(self, x):
        """
        Returns the penultimate embedding (for FID).
        """
        h = self.conv(x)
        h = h.view(h.size(0), -1)
        h = self.fc_embed(h)
        return h

    def forward(self, x):
        """
        Returns class logits.
        """
        h = self.forward_features(x)
        logits = self.fc_out(F.relu(h))
        return logits


In [ ]:
MNIST_CLASSIFER_PATH = BOERMA_MODEL_DIR / "classifer_weights.pt"

In [ ]:
### Cell (mostly) generated by AI
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MNISTClassifier(embedding_dim=128).to(device)
if MNIST_CLASSIFER_PATH.exists(): 
    weights = torch.load(MNIST_CLASSIFER_PATH)
    model.load_state_dict(weights)
else: 
    transform = transforms.ToTensor()

    train_ds = datasets.MNIST(
        root=DATASET_PATH,
        train=True,
        download=True,
        transform=transform,
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)

    model = MNISTClassifier(embedding_dim=128).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(3):  # MNIST converges fast
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}: loss={loss.item():.4f}")

    torch.save(model.state_dict(), MNIST_CLASSIFER_PATH)

Epoch 0: loss=0.0414
Epoch 1: loss=0.0533
Epoch 2: loss=0.0030


In [ ]:
@torch.no_grad()
def extract_embeddings(model, dataloader, device):
    model.eval()
    embeddings = []

    for x, _ in dataloader:
        x = x.to(device)
        emb = model.forward_features(x)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)

### MNIST-Wass

### MNIST-MMD

### CIFAR-10-FID